# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')
y_val = pd.read_csv('data/y_val.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis = 1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis = 1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis = 1)

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

Here, we're calling .shape on our training data so that we can use the result as n_features, so we know how big to make our input layer.

In [5]:
# How big input layer?
n_features = (X_train.shape[1],)
print(n_features)

(296,)


Create your baseline model. Yo will notice is exihibits strange behavior.

*Note:* When you run this model or other models later on, you may get a comment from tf letting you about optimizing your GPU

In [6]:
# Baseline model
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation = 'relu', input_shape = (n_features)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation = 'relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation = 'linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 10/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - l

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [7]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [8]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 
Again, you may get some strange behavior.

In [9]:
# Train the model
normalized_input_model.fit(X_train, y_train, batch_size = 32, epochs = 150, validation_data = (X_val, y_val))

Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: inf - mse: inf - val_loss: inf - val_mse: inf
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: inf - mse: inf - val_loss: inf - val_mse: inf
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: inf - mse: inf - val_loss: inf - val_mse: inf
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: inf - mse: inf - val_loss: inf - val_mse: inf
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: inf - mse: inf - val_loss: inf - val_mse: inf
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: inf - mse: inf - val_loss: inf - val_mse: inf
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: inf - mse: inf - val_loss: inf - val_mse: inf
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: inf - mse: inf - val_loss: inf - val_mse: inf
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: inf - mse: inf - val_loss: inf - val_mse: inf
Epoch 10/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - l

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [10]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [12]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, y_train_scaled, batch_size = 32, epochs = 150, validation_data = (X_val, y_val_scaled))

Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5028 - mse: 0.5028 - val_loss: 0.1912 - val_mse: 0.1912
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2439 - mse: 0.2439 - val_loss: 0.1467 - val_mse: 0.1467
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1921 - mse: 0.1921 - val_loss: 0.1358 - val_mse: 0.1358
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1303 - mse: 0.1303 - val_loss: 0.1334 - val_mse: 0.1334
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1552 - mse: 0.1552 - val_loss: 0.1295 - val_mse: 0.1295
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1129 - mse: 0.1129 - val_loss: 0.1203 - val_mse: 0.1203
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1390 - mse: 0.1390 - val_loss: 0.1231 - val_mse: 0.1231
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1342 - mse: 0.1342 - val_loss: 0.1185 - val_mse: 0.1185
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.091

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [13]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 594us/step - loss: 0.0140 - mse: 0.0140


[0.010891425423324108, 0.010891425423324108]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [14]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 750us/step - loss: 0.1068 - mse: 0.1068


[0.11321596801280975, 0.11321596801280975]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [15]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


26441.454034097653

Great. Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'` 
  
[Documentation on the He Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/HeNormal)

In [16]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, kernel_initializer='he_normal', activation='relu', input_shape=(n_features)))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', loss='mse', metrics=['mse'])
# Train the model
he_model.fit(X_train, y_train_scaled, batch_size = 32, epochs = 150, validation_data = (X_val, y_val_scaled))

Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7357 - mse: 0.7357 - val_loss: 0.2365 - val_mse: 0.2365
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2778 - mse: 0.2778 - val_loss: 0.1725 - val_mse: 0.1725
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1834 - mse: 0.1834 - val_loss: 0.1477 - val_mse: 0.1477
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2014 - mse: 0.2014 - val_loss: 0.1528 - val_mse: 0.1528
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1627 - mse: 0.1627 - val_loss: 0.1358 - val_mse: 0.1358
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1473 - mse: 0.1473 - val_loss: 0.1318 - val_mse: 0.1318
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1243 - mse: 0.1243 - val_loss: 0.1497 - val_mse: 0.1497
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1388 - mse: 0.1388 - val_loss: 0.1245 - val_mse: 0.1245
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.109

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [17]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 594us/step - loss: 0.0107 - mse: 0.0107


[0.009048104286193848, 0.009048104286193848]

Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`)

In [18]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 875us/step - loss: 0.1055 - mse: 0.1055


[0.1186814084649086, 0.1186814084649086]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 
  
[Documentation on the Lecun Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/LecunNormal)

In [19]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=n_features))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', loss='mse', metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.8257 - mse: 0.8257 - val_loss: 0.1732 - val_mse: 0.1732
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2824 - mse: 0.2824 - val_loss: 0.1400 - val_mse: 0.1400
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1747 - mse: 0.1747 - val_loss: 0.1236 - val_mse: 0.1236
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1462 - mse: 0.1462 - val_loss: 0.1169 - val_mse: 0.1169
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1322 - mse: 0.1322 - val_loss: 0.1139 - val_mse: 0.1139
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1475 - mse: 0.1475 - val_loss: 0.1119 - val_mse: 0.1119
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1030 - mse: 0.1030 - val_loss: 0.1087 - val_mse: 0.1087
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1175 - mse: 0.1175 - val_loss: 0.1063 - val_mse: 0.1063
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.080

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [20]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 594us/step - loss: 0.0083 - mse: 0.0083


[0.0070716990157961845, 0.0070716990157961845]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [21]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 750us/step - loss: 0.1226 - mse: 0.1226


[0.11818528175354004, 0.11818528175354004]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

[Documentation on the RMS Prop Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/RMSprop)

In [25]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
# Code here
rmsprop_model.compile(optimizer='rmsprop', loss='mse', metrics=['mse'])
# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.5274 - mse: 0.5274 - val_loss: 0.1109 - val_mse: 0.1109
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1612 - mse: 0.1612 - val_loss: 0.1026 - val_mse: 0.1026
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1236 - mse: 0.1236 - val_loss: 0.1368 - val_mse: 0.1368
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0916 - mse: 0.0916 - val_loss: 0.1206 - val_mse: 0.1206
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0899 - mse: 0.0899 - val_loss: 0.0983 - val_mse: 0.0983
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0706 - mse: 0.0706 - val_loss: 0.0988 - val_mse: 0.0988
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0762 - mse: 0.0762 - val_loss: 0.1315 - val_mse: 0.1315
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0499 - mse: 0.0499 - val_loss: 0.0954 - val_mse: 0.0954
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.039

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [26]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 627us/step - loss: 0.0025 - mse: 0.0025


[0.0027674022130668163, 0.0027674022130668163]

Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`) 

In [27]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 875us/step - loss: 0.0768 - mse: 0.0768


[0.07362983375787735, 0.07362983375787735]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric

[Documentation on the Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)

In [28]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
# Code here
adam_model.compile(optimizer='Adam', loss='mse', metrics=['mse'])
# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.6227 - mse: 0.6227 - val_loss: 0.1218 - val_mse: 0.1218
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1997 - mse: 0.1997 - val_loss: 0.0903 - val_mse: 0.0903
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0976 - mse: 0.0976 - val_loss: 0.0880 - val_mse: 0.0880
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0940 - mse: 0.0940 - val_loss: 0.0861 - val_mse: 0.0861
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0582 - mse: 0.0582 - val_loss: 0.0870 - val_mse: 0.0870
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0604 - mse: 0.0604 - val_loss: 0.0851 - val_mse: 0.0851
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0438 - mse: 0.0438 - val_loss: 0.0940 - val_mse: 0.0940
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0316 - mse: 0.0316 - val_loss: 0.0940 - val_mse: 0.0940
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.026

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [29]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 778us/step - loss: 0.0011 - mse: 0.0011


[0.0014515717048197985, 0.0014515717048197985]

Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`) 

In [30]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 875us/step - loss: 0.0838 - mse: 0.0838


[0.08797520399093628, 0.08797520399093628]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [31]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1289 - mse: 0.1289 


[0.1617213487625122, 0.1617213636636734]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [33]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


31602.046017231107

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.